# Transcripción Automática de Audio con Whisper

**Propósito:** Transcribir archivos de audio a texto de forma simple y rápida.

**Modelo:** OpenAI Whisper

---

## ¿Qué vamos a hacer?

En este cuaderno vas a aprender a usar Whisper, el modelo de OpenAI para reconocimiento de voz, para convertir archivos de audio en texto.

**Casos de uso:**
- Transcribir reuniones y entrevistas
- Convertir podcasts a texto
- Generar subtítulos para videos
- Documentar conferencias y charlas

**Requisitos:**
- Google Colab con GPU habilitada (Runtime > Change runtime type > GPU)
- Archivo de audio en formato común (MP3, WAV, M4A, etc.)

## Paso 1: Instalación de Dependencias

Instalamos las librerías necesarias para trabajar con Whisper.

In [1]:
# Instalar dependencias
!pip install -U -q transformers torch torchvision accelerate gradio librosa

print("✅ Dependencias instaladas correctamente")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 75.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.3/322.3 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.3/39.3 MB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124

## Paso 2: Importar Librerías y Configurar el Modelo

Importamos las librerías y configuramos el modelo de Whisper que vamos a usar.

In [2]:
import torch
from transformers import pipeline
import gradio as gr
import warnings

warnings.filterwarnings('ignore')

print("📦 Librerías importadas correctamente")

📦 Librerías importadas correctamente


### Selección del Modelo

Whisper tiene diferentes tamaños de modelos. Elegí según tus necesidades:

| Modelo | Tamaño | Velocidad | Precisión | Uso recomendado |
|--------|--------|-----------|-----------|------------------|
| tiny | ~39 MB | Muy rápido | Básica | Pruebas rápidas |
| base | ~74 MB | Rápido | Buena | Audio claro |
| small | ~244 MB | Moderado | Muy buena | **Recomendado para empezar** |
| medium | ~769 MB | Lento | Excelente | Audio con ruido |
| large-v3 | ~1550 MB | Muy lento | Máxima | Máxima calidad |

In [3]:
# Seleccionar modelo (cambia aquí si quieres probar otro)
MODELO_WHISPER = "openai/whisper-small"  # Cambia a tiny, base, medium o large-v3

print(f"🤖 Modelo seleccionado: {MODELO_WHISPER}")

🤖 Modelo seleccionado: openai/whisper-small


## Paso 3: Cargar el Modelo de Whisper

Cargamos el modelo en la GPU si está disponible.

In [4]:
# Detectar dispositivo (GPU o CPU)
device = 0 if torch.cuda.is_available() else -1
torch_dtype = torch.bfloat16 if torch.cuda.is_available() else torch.float32

print(f"💻 Dispositivo: {'GPU ✅' if device == 0 else 'CPU ⚠️'}")
if device == -1:
    print("⚠️ No hay GPU disponible. La transcripción será más lenta.")
    print("   Para usar GPU: Runtime > Change runtime type > GPU")

# Cargar modelo
print(f"\n🔄 Cargando modelo {MODELO_WHISPER}...")
print("   (Esto puede tardar unos minutos la primera vez)\n")

transcriptor = pipeline(
    "automatic-speech-recognition",
    model=MODELO_WHISPER,
    device=device,
    torch_dtype=torch_dtype,
    return_timestamps=True,
    chunk_length_s=30,  # Procesa en chunks de 30 segundos
    stride_length_s=5   # Superposición de 5 segundos
)

print("✅ Modelo cargado exitosamente\n")
print("🎯 Listo para transcribir audio")

💻 Dispositivo: GPU ✅

🔄 Cargando modelo openai/whisper-small...
   (Esto puede tardar unos minutos la primera vez)



config.json: 0.00B [00:00, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

...

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

preprocessor_config.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0
Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).


✅ Modelo cargado exitosamente

🎯 Listo para transcribir audio


## Paso 4: Función de Transcripción

Creamos una función simple que toma un archivo de audio y retorna su transcripción.

In [6]:
def transcribir_audio(archivo_audio):
    """
    Transcribe un archivo de audio a texto usando Whisper.

    Args:
        archivo_audio: Ruta al archivo de audio

    Returns:
        str: Transcripción del audio
    """
    if not archivo_audio:
        return "❌ No se cargó ningún archivo de audio"

    try:
        print("🎯 Iniciando transcripción...")

        # Transcribir
        resultado = transcriptor(archivo_audio)

        # Extraer texto
        texto = ""
        if isinstance(resultado, dict) and "text" in resultado:
            texto = resultado["text"].strip()
        elif isinstance(resultado, str):
            texto = resultado.strip()

        if not texto:
            return "❌ No se pudo extraer texto del audio"

        print(f"✅ Transcripción completada: {len(texto)} caracteres")

        # Formatear salida
        salida = f"📝 TRANSCRIPCIÓN\n"
        salida += f"{'='*60}\n\n"
        salida += texto
        salida += f"\n\n{'='*60}\n"
        salida += f"📊 Total de caracteres: {len(texto)}\n"
        salida += f"📊 Palabras aproximadas: {len(texto.split())}\n"
        salida += f"🤖 Modelo utilizado: {MODELO_WHISPER}"

        return salida

    except Exception as e:
        error = f"❌ Error durante la transcripción:\n{str(e)}"
        print(error)
        return error

print("✅ Función de transcripción definida")

✅ Función de transcripción definida


## Paso 5: Interfaz Interactiva con Gradio

Creamos una interfaz web simple para subir archivos y ver las transcripciones.

In [7]:
# Crear interfaz
interfaz = gr.Interface(
    fn=transcribir_audio,
    inputs=gr.Audio(
        sources=["upload"],
        type="filepath",
        label="🎵 Subir Archivo de Audio"
    ),
    outputs=gr.Textbox(
        label="📝 Transcripción",
        lines=15,
        show_copy_button=True,
        placeholder="La transcripción aparecerá aquí..."
    ),
    title="🎙️ Transcriptor de Audio con Whisper",
    description=f"""
    ### 🚀 Transcripción automática de audio a texto

    **Modelo:** {MODELO_WHISPER}
    **Dispositivo:** {'GPU' if device == 0 else 'CPU'}

    #### 📋 Instrucciones:
    1. Sube tu archivo de audio (MP3, WAV, M4A, etc.)
    2. Espera a que se procese
    3. Copia o descarga la transcripción

    #### ✨ Características:
    - Soporta múltiples idiomas (español, inglés, etc.)
    - Maneja archivos de cualquier duración
    - Detecta automáticamente el idioma
    - Alta precisión de transcripción
    """,
    examples=[],
    cache_examples=False,
    allow_flagging="never",
    theme=gr.themes.Soft()
)

print("✅ Interfaz creada")

✅ Interfaz creada


## Paso 6: Lanzar la Aplicación

Ejecuta esta celda para iniciar la interfaz web.

In [8]:
# Lanzar aplicación
print("🚀 Lanzando aplicación...\n")

interfaz.launch(
    share=True,      # URL pública para compartir
    debug=True,      # Mostrar información de debug
    show_error=True  # Mostrar errores en la interfaz
)

print("""
✅ Aplicación en ejecución

💡 Consejos:
- Para mejor precisión, usa audio claro y sin mucho ruido de fondo
- Si el audio es muy largo, puede tardar varios minutos
- Podes copiar la transcripción con el botón de copiar

⚠️ Importante:
- Esta aplicación se ejecuta mientras esta celda esté corriendo
- Para detenerla, presiona el botón de stop en esta celda
""")

🚀 Lanzando aplicación...

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://0f8286b206f86c44c6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


🎯 Iniciando transcripción...


Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
Transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English. This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`. See https://github.com/huggingface/transformers/pull/28687 for more details.


✅ Transcripción completada: 83 caracteres
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://0f8286b206f86c44c6.gradio.live

✅ Aplicación en ejecución

💡 Consejos:
- Para mejor precisión, usa audio claro y sin mucho ruido de fondo
- Si el audio es muy largo, puede tardar varios minutos
- Podes copiar la transcripción con el botón de copiar

⚠️ Importante:
- Esta aplicación se ejecuta mientras esta celda esté corriendo
- Para detenerla, presiona el botón de stop en esta celda



---

## Opción Alternativa: Transcribir Directamente desde Código

Si preferís transcribir sin usar la interfaz web, podes usar estas celdas:

In [9]:
# Subir archivo manualmente
from google.colab import files

print("📁 Selecciona tu archivo de audio:")
archivo_subido = files.upload()

# Obtener nombre del archivo
nombre_archivo = list(archivo_subido.keys())[0]
print(f"\n✅ Archivo cargado: {nombre_archivo}")

📁 Selecciona tu archivo de audio:


Saving Grabación.mp3 to Grabación.mp3

✅ Archivo cargado: Grabación.mp3


In [10]:
# Transcribir el archivo
resultado = transcribir_audio(nombre_archivo)
print(resultado)

🎯 Iniciando transcripción...
✅ Transcripción completada: 83 caracteres
📝 TRANSCRIPCIÓN

Esta es una grabación de prueba para la materia procesamiento del lenguaje natural.

📊 Total de caracteres: 83
📊 Palabras aproximadas: 13
🤖 Modelo utilizado: openai/whisper-small


In [11]:
# Opcional: Guardar transcripción en archivo
nombre_salida = nombre_archivo.rsplit('.', 1)[0] + '_transcripcion.txt'

with open(nombre_salida, 'w', encoding='utf-8') as f:
    f.write(resultado)

print(f"💾 Transcripción guardada en: {nombre_salida}")

# Descargar el archivo
files.download(nombre_salida)

💾 Transcripción guardada en: Grabación_transcripcion.txt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

---

## Ejercicios Prácticos

### Ejercicio 1: Comparar Modelos
1. Transcribe el mismo audio con diferentes modelos (tiny, small, medium)
2. Compara la precisión y el tiempo de procesamiento
3. ¿Cuál es el mejor balance entre velocidad y calidad?

### Ejercicio 2: Audio con Ruido
1. Prueba con audios de diferentes calidades
2. ¿Cómo afecta el ruido de fondo a la transcripción?
3. ¿Qué modelo funciona mejor con audio ruidoso?

### Ejercicio 3: Múltiples Idiomas
1. Transcribe audios en diferentes idiomas
2. ¿Whisper detecta correctamente el idioma?
3. ¿La precisión varía según el idioma?

### Ejercicio 4: Procesamiento por Lotes
Modifica el código para transcribir múltiples archivos de una vez.

---

## Solución de Problemas

### Error: Out of Memory
- Usa un modelo más pequeño (tiny o base)
- Reduce el tamaño de chunk_length_s
- Reinicia el runtime y limpia la memoria

### Transcripción Imprecisa
- Usa un modelo más grande (medium o large-v3)
- Mejora la calidad del audio
- Reduce el ruido de fondo

### Proceso Muy Lento
- Verifica que estés usando GPU
- Usa un modelo más pequeño
- Divide el audio en partes más cortas

---

## Recursos Adicionales

- [Whisper GitHub](https://github.com/openai/whisper)
- [Whisper en Hugging Face](https://huggingface.co/openai/whisper-small)
- [Documentación de Transformers](https://huggingface.co/docs/transformers/)

---

**Creado para el curso de Procesamiento del Lenguaje Natural**  
**Última actualización:** Noviembre 2025